In [56]:
import requests
import json
from bs4 import BeautifulSoup

### Coin class
Create a coin class that will keep track of each coin's data, such as url, coin name and the images_url

In [75]:
# Class for each coin category.
class Coin:
    def __init__(self, coin_id, coin_name, coin_currency, coin_country):
        self.id = coin_id
        self.name = coin_name
        self.currency = coin_currency
        self.country = coin_country
        self.currency_url = "" # Url of the web we are going to scrap.
        self.image_urls = [] # All the relevant images urls ready to download.
        self.soup = None # Our BeautifulSoup containing the html object.
        
    # Return the url we are going to scrap
    def get_url (self, year="2006-2018"):
        base_url = "https://en.ucoin.net/catalog/?" # Website url
        country = f"country={self.country}" # Parameters
        year = f"year={year}" # Parameters

        url = base_url + country + "&" + year + "&type=1"
        
        self.currency_url = url # Update our currency_url variable.
    
    # Store the html of the web in a variable.
    def get_html (self):
        page = requests.get(self.currency_url)
        
        if page.status_code == 200:
            self.soup = BeautifulSoup(page.content, 'html.parser')
        else:
            raise ValueError(f"Status code {page.status_code} for url: {self.currency_url}") # Throw error if 404
    
    # Examine the html to find images of our specific coin.
    def get_image_urls (self):
        pass

### Create a list of Coin instances

Create a list with all the necessary data for each coin to start scrapping.

In [76]:
# Create a dict containing the json.
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Check that it worked
cat_to_name["1"].split(",")

['1 Cent', 'Australian dollar', 'australia']

In [77]:
# A list containing an instance of Coin for each coin.
coin_list = []

# Append coins to coin_list.
for coin in cat_to_name.items():
    # Get the id, name, currency and country
    coin_id = coin[0]
    coin_data = coin[1].split(",")
    coin_name, coin_currency, coin_country = coin_data[0], coin_data[1], coin_data[2]
    
    # Create a Coin instance.
    my_coin = Coin(coin_id, coin_name, coin_currency, coin_country)
    # Push it into coin list.
    coin_list.append(my_coin)